In [21]:
import pandas as pd
import numpy as np
import codecs
import sys
import nltk
from bs4 import BeautifulSoup
import re
import gensim
import string

from IPython.display import clear_output

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk import word_tokenize
import spacy
import emot
from emot.emo_unicode import EMOTICONS_EMO
from spellchecker import SpellChecker
from translate import Translator
import translators as ts
import translators.server as tss
#from emot.emo_unicode import UNICODE_EMO, EMOTICONS

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lukasburtscher/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukasburtscher/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
doc = codecs.open('../data/coronaPostings.csv','rU','UTF-8') #open for reading with "universal" type set

df = pd.read_csv(doc, sep='\t')
print(df.shape)
df.head()

(13151, 10)


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive
0,1051800287,NaN,2000116189812,184470,1.0,Nicht sinnvoll,"Forschungen, Studien, Statistiken unddieganze ...",2020-03-26 16:09:27.953,True,False
1,1051801262,NaN,2000116189812,240940,1.0,super,erst ganz unten steht das wichtigste,NaN,NaN,NaN
2,hält von den Schlussfolgerungen aus den Simula...,2020-03-26 16:28:10.400,True,False,NaN,NaN,NaN,NaN,NaN,NaN
3,1054666961,1054666937,2000117671399,602300,1.0,NaN,es handelt sich um 1 Pressekonferenz. Mit Paus...,2020-05-25 09:19:27.010,False,True
4,1054667707,NaN,2000117671399,56623,1.0,NaN,die DB app sollte vor allem eines: gut funktio...,NaN,NaN,NaN


In [3]:
##drop missformated rows where values are NaN (excluding ID_Posting_Parent)
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.dropna(thresh=8)
df = df.dropna(subset=["Comment"])
df['createdAt'] = pd.to_datetime(df['createdAt'])
df.ID_GodotObject = df.ID_GodotObject.astype('int64')
df.ID_Posting = df.ID_Posting.astype('int64')
df["ID_Posting_Parent"] = df["ID_Posting_Parent"].fillna(-1)
df.ID_Posting_Parent = df.ID_Posting_Parent.astype('int64')
df = df.sort_values(by='createdAt').reset_index(drop=True)
df = df.drop_duplicates(subset=["ID_Posting"])
print(df.shape)
print(df.dtypes)
df.head()

(6485, 10)
ID_Posting                       int64
ID_Posting_Parent                int64
ID_GodotObject                   int64
ID_CommunityIdentity            object
id_posting_status              float64
Headline                        object
Comment                         object
createdAt               datetime64[ns]
VoteNegative                    object
VotePositive                    object
dtype: object


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive
0,1049746955,1049746300,2000114365338,581535,1.0,NaN,"Nö, noch um einiges schlimmer.",2020-02-10 13:39:17.460,False,True
1,1051784424,1051784054,2000116189812,632068,1.0,NaN,"Wieder einer, der nicht kapiert, dass Arbeit k...",2020-03-26 12:08:22.187,False,True
2,1051784801,-1,2000116189812,600252,1.0,ich fasse es mit den worten von prof. falco zu...,nie mehr schule!,2020-03-26 12:13:29.653,True,False
4,1051785187,-1,2000116189812,701802,1.0,NaN,Das wird Monate dauern ihr Traumtänzer... Das ...,2020-03-26 12:19:13.047,True,False
5,1051785575,1051784986,2000116189812,702957,1.0,Dann sind wir gefordert,und müssen in Massen auf dieStrassen.,2020-03-26 12:25:26.320,False,True


In [48]:
##find the maximal deph in the forumnsposting

def find_maximum_depth(posting_id, depth):
    # find all direct replies for the posting
    #print(posting_id)
    replies = df[df["ID_Posting_Parent"] == posting_id]
    #print(len(replies))
    if replies.empty:
        # Return the current depth if there are no replies
        return depth
    else:
        current_depth = depth
        for row in replies.itertuples():
            current_depth = max(current_depth, find_maximum_depth(row.ID_Posting, depth+1))
        return current_depth
    
    
    
# Converting emoticons to words and track emojis that were replaced
emojis = [':D', ':3', '%)', ':(', ';D', '8-)', 'XD', ':-)', ':*', ':)', 'd:', ':O', ':P', ':/', ':-))', ":')", '=L', ';)', ':-*', ';_;', ':o', ':-(', '=D']
EMOTICONS_DE = {
    u":D":"Lachen",
    u":3":"glückliches Gesicht",
    u"%)":"betrunken und verwirrt",
    u":(":"traurig",
    u";D":"zwinkernd lachend",
    u"8-)":"glücklich",
    u"XD":"witzig",
    u":-)":"zufrieden",
    u":*":"Kuss",
    u":)":"zufrieden",
    u"d:":"spielend",
    u":O":"schockiert",
    u":P":"spielend",
    u":/":"mittelmäßig",
    u":-))":"sehr_zufrieden",
    u":')":"Freudentränen",
    u"=L":"spielend",
    u";)":"Zwinker",
    u":-*":"Kuss",
    u":o":"schockiert",
    u":-(":"traurig",
    u"=D":"glücklich",
}
def convert_emoticons(text):
    for emoticon, description in EMOTICONS_DE.items():
        if emoticon in text:
            emojis.append(emoticon)
            text = text.replace(emoticon, description)
    return text

In [49]:
#find_maximum_depth(1051784054, 1)

m_depth = max([(posting_id,find_maximum_depth(posting_id, 1)) for posting_id in df['ID_Posting']],key=lambda x: x[1])
m_depth

(1052015173, 2)

In [50]:
#emoticons preprocessing with translation
emot_obj = emot.core.emot()
text = "Ich wills net zugebn ☮ 🙂 ❤ :-) ;) :-( :-)))" 
print(list(set(emojis)))
convert_emoticons(text)


[':D', ':3', '%)', ':(', '8-)', ';D', 'XD', ':-)', ':*', ':)', 'd:', ':O', ':P', ':/', ':-))', ":')", '=L', ';)', ':-*', ';_;', ':o', ':-(', '=D']


'Ich wills net zugebn ☮ 🙂 ❤ zufrieden Zwinker traurig zufrieden))'

In [9]:
# spell correction for austrian language
text = 'wos kann ich tun für dich ? happy face smiling'
spell = SpellChecker(language='de')
misspelled = spell.unknown(text.split())
corrected_text = text
for word in misspelled:
    corrected_text = corrected_text.replace(word, spell.correction(word))
    
print(corrected_text)

was kann ich tun für dich ? happy face shining


In [10]:
# translate english to german
translator= Translator(to_lang="German")
translation = translator.translate(corrected_text)
print(translation)

print(tss.google(corrected_text, from_language='en', to_language='de'))

Was kann ich tun?
War Kann ich Tun für für Dich? Glückliches Gesicht scheint


In [34]:
# preprocessing pipeline
counter = 0
def preprocessing(text):
    text = str(text)
    #transform emojis
    text = convert_emoticons(text)
    #spell correction
    misspelled = spell.unknown(text.split())
    corrected_text = text
    for word in misspelled:
        if spell.correction(word) is not None:
            corrected_text = corrected_text.replace(word, spell.correction(word))
    text = corrected_text
    
    
    #translate everything to german
    #text = tss.google(text, from_language='en', to_language='de')
    #lowercase-punctuation-stopwords
    tokens = nltk.word_tokenize(text)
    lowercase_tokens = [token.lower() for token in tokens]
    clean_tokens = [re.sub(r'[^\w\s]', '', token) for token in lowercase_tokens]
    stop_words = stopwords.words("german")
    filtered_words = [w for w in clean_tokens if w not in stop_words and w]
    #return filtered_words
    
    # Lemmatization
    nlp = spacy.load('de_core_news_md')
    filtered_lemma = []
    for w in filtered_words:
        result = ' '.join([token.lemma_ for token in nlp(w)]) 
        filtered_lemma.append(result)
        
    clear_output()
    # increment the counter
    global counter
    counter += 1
    print(counter)
    return ' '.join(filtered_lemma)
    
    
    

In [35]:
df1 = df.copy()
df1['p_comment'] = df1['Comment'].apply(lambda x: preprocessing(x))
df1['p_headline'] = df1['Headline'].apply(lambda x: preprocessing(x))


#all emojis in the text
print(list(set(emojis)))

df1.head()

12970
[':D', ':3', '%)', ':(', ';D', '8-)', 'XD', ':-)', ':*', ':)', 'd:', ':O', ':P', ':/', ':-))', ":')", '=L', ';)', ':-*', ';_;', ':o', ':-(', '=D']


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive,p_comment,p_headline
0,1049746955,1049746300,2000114365338,581535,1.0,NaN,"Nö, noch um einiges schlimmer.",2020-02-10 13:39:17.460,False,True,nö schlimm,
1,1051784424,1051784054,2000116189812,632068,1.0,NaN,"Wieder einer, der nicht kapiert, dass Arbeit k...",2020-03-26 12:08:22.187,False,True,kapieren Arbeit Gängelei notwendig Versorgung ...,
2,1051784801,-1,2000116189812,600252,1.0,ich fasse es mit den worten von prof. falco zu...,nie mehr schule!,2020-03-26 12:13:29.653,True,False,nie mehr Schule,fasse warten pro zusammen
4,1051785187,-1,2000116189812,701802,1.0,NaN,Das wird Monate dauern ihr Traumtänzer... Das ...,2020-03-26 12:19:13.047,True,False,Monat dauern Traumtänzer stehen einfach dafür,
5,1051785575,1051784986,2000116189812,702957,1.0,Dann sind wir gefordert,und müssen in Massen auf dieStrassen.,2020-03-26 12:25:26.320,False,True,müssen Masse Diestrassen,fordern


In [51]:
#merging content with title and build tokens
df2 = df1.drop(['Comment','Headline'],axis=1)
df2.rename(columns = {"p_comment":"comment","p_headline":"headline"}, inplace = True)
text_columns = ["comment", "headline"]
df2["merged_text"] = df2[text_columns].apply(lambda x: " | ".join(x), axis=1)
df2["tokens"] = df2["merged_text"].map(lambda x: x.split())
df2.head()

,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,createdAt,VoteNegative,VotePositive,comment,headline,merged_text,tokens
0,1049746955,1049746300,2000114365338,581535,1.0,2020-02-10 13:39:17.460,False,True,nö schlimm,,nö schlimm |,"[nö, schlimm, |]"
1,1051784424,1051784054,2000116189812,632068,1.0,2020-03-26 12:08:22.187,False,True,kapieren Arbeit Gängelei notwendig Versorgung ...,,kapieren Arbeit Gängelei notwendig Versorgung ...,"[kapieren, Arbeit, Gängelei, notwendig, Versor..."
2,1051784801,-1,2000116189812,600252,1.0,2020-03-26 12:13:29.653,True,False,nie mehr Schule,fasse warten pro zusammen,nie mehr Schule | fasse warten pro zusammen,"[nie, mehr, Schule, |, fasse, warten, pro, zus..."
4,1051785187,-1,2000116189812,701802,1.0,2020-03-26 12:19:13.047,True,False,Monat dauern Traumtänzer stehen einfach dafür,,Monat dauern Traumtänzer stehen einfach dafür |,"[Monat, dauern, Traumtänzer, stehen, einfach, ..."
5,1051785575,1051784986,2000116189812,702957,1.0,2020-03-26 12:25:26.320,False,True,müssen Masse Diestrassen,fordern,müssen Masse Diestrassen | fordern,"[müssen, Masse, Diestrassen, |, fordern]"


In [52]:
##building phrases from all comments
tokens = df2["tokens"].tolist()
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokens, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokens], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


data_words = make_trigrams(tokens)
data_words_join = [" ".join(inner) for inner in data_words]
df2["merged_text"] = np.array(data_words_join)
df2["tokens"] = np.array(data_words)
df2.head()

/var/folders/1_/83qhk9ps6fd0yym23nn9nfz00000gn/T/ipykernel_6713/1330523528.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df2["tokens"] = np.array(data_words)


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,createdAt,VoteNegative,VotePositive,comment,headline,merged_text,tokens
0,1049746955,1049746300,2000114365338,581535,1.0,2020-02-10 13:39:17.460,False,True,nö schlimm,,nö schlimm |,"[nö, schlimm, |]"
1,1051784424,1051784054,2000116189812,632068,1.0,2020-03-26 12:08:22.187,False,True,kapieren Arbeit Gängelei notwendig Versorgung ...,,kapieren Arbeit Gängelei notwendig Versorgung ...,"[kapieren, Arbeit, Gängelei, notwendig, Versor..."
2,1051784801,-1,2000116189812,600252,1.0,2020-03-26 12:13:29.653,True,False,nie mehr Schule,fasse warten pro zusammen,nie mehr Schule | fasse warten pro zusammen,"[nie, mehr, Schule, |, fasse, warten, pro, zus..."
4,1051785187,-1,2000116189812,701802,1.0,2020-03-26 12:19:13.047,True,False,Monat dauern Traumtänzer stehen einfach dafür,,Monat dauern Traumtänzer stehen einfach dafür |,"[Monat, dauern, Traumtänzer, stehen, einfach, ..."
5,1051785575,1051784986,2000116189812,702957,1.0,2020-03-26 12:25:26.320,False,True,müssen Masse Diestrassen,fordern,müssen Masse Diestrassen | fordern,"[müssen, Masse, Diestrassen, |, fordern]"


In [54]:
df2.to_csv('../data/p_comment.csv', encoding='utf-8', index=False)